<a href="https://colab.research.google.com/github/nightmarewarrior/homework/blob/main/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
!pip install razdel
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML

In [2]:
import pandas as pd

In [3]:
! wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/notebooks/bow/labeled.csv.zip
!unzip -o labeled.csv.zip

data = pd.read_csv('labeled.csv')
data.head()

--2024-11-24 20:21:29--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/notebooks/bow/labeled.csv.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/bow/labeled.csv.zip [following]
--2024-11-24 20:21:29--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/bow/labeled.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1463222 (1.4M) [application/zip]
Saving to: ‘labeled.csv.zip’

labeled.csv.zip     100%[===================>]   1.39M  --.-KB/s    in 0.07s   

2024-11-24 20:21:29 (20.1 M

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from razdel import tokenize


train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

#Модель с дефолтной токенизацией

vectorizer_default = CountVectorizer(ngram_range=(1, 2))
X_train_default = vectorizer_default.fit_transform(train.comment)
X_test_default = vectorizer_default.transform(test.comment)

#Обучение классификатора (логическая регрессия)
clf_default_lr = LogisticRegression()
clf_default_lr.fit(X_train_default, train.toxic.values)

y_pred_default_lr = clf_default_lr.predict(X_test_default)
print("Дефолтная токенизация (LR):")
print(classification_report(test.toxic.values, y_pred_default_lr))




Дефолтная токенизация (LR):
              precision    recall  f1-score   support

         0.0       0.85      0.95      0.90       969
         1.0       0.86      0.66      0.75       473

    accuracy                           0.85      1442
   macro avg       0.86      0.81      0.82      1442
weighted avg       0.85      0.85      0.85      1442



In [5]:
# --- Модель с кастомной токенизацией (razdel.tokenize) ---

def razdel_tokenizer(text):
    return [token.text for token in tokenize(text)]


vectorizer_custom = CountVectorizer(
    tokenizer=razdel_tokenizer,
    ngram_range=(1, 2)
    )
X_train_custom = vectorizer_custom.fit_transform(train.comment)
X_test_custom = vectorizer_custom.transform(test.comment)


clf_custom = LogisticRegression()
clf_custom.fit(X_train_custom, train.toxic.values)

y_pred_custom = clf_custom.predict(X_test_custom)
print("Кастомная токенизация (razdel):")
print(classification_report(test.toxic.values, y_pred_custom))


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Кастомная токенизация (razdel):
              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90       969
         1.0       0.85      0.71      0.77       473

    accuracy                           0.86      1442
   macro avg       0.86      0.82      0.84      1442
weighted avg       0.86      0.86      0.86      1442



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


С дефолтной токенизацией выдаются результаты лучше.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [23]:
#KNN
russian_stopwords = stopwords.words("russian")

train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

vectorizer = TfidfVectorizer(stop_words=russian_stopwords, min_df=1, max_df=0.7, use_idf=True, smooth_idf=True, sublinear_tf=False, ngram_range=(1, 1))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

clf = KNeighborsClassifier(n_neighbors=15, weights='distance', algorithm='ball_tree', leaf_size=100, metric='minkowski')
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_base.py:583: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


              precision    recall  f1-score   support

         0.0       0.85      0.93      0.89       953
         1.0       0.82      0.68      0.75       489

    accuracy                           0.84      1442
   macro avg       0.84      0.80      0.82      1442
weighted avg       0.84      0.84      0.84      1442



In [24]:
#NB

vectorizer = CountVectorizer(stop_words=russian_stopwords, ngram_range=(1, 1), analyzer='word', max_df=0.4, min_df=5)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

clf = MultinomialNB(alpha=1.0, force_alpha=True, fit_prior=False, class_prior=None)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))


              precision    recall  f1-score   support

         0.0       0.90      0.88      0.89       953
         1.0       0.77      0.81      0.79       489

    accuracy                           0.86      1442
   macro avg       0.84      0.84      0.84      1442
weighted avg       0.86      0.86      0.86      1442



## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [34]:
import numpy as np

top_toxic_words = {}

# 1. Logistic Regression
log_reg = LogisticRegression(C=1.0, max_iter=500)
log_reg.fit(X, y)
y_pred_log_reg = log_reg.predict(X_test)
print("Logistic Regression:")
print(classification_report(y_test, y_pred_log_reg))


feature_names = vectorizer.get_feature_names_out()
coefs = log_reg.coef_[0]
toxic_features_log_reg = sorted(zip(coefs, feature_names), reverse=True)[:5]
top_toxic_words['Logistic Regression'] = [word for coef, word in toxic_features_log_reg]

# Decision Trees
dt = DecisionTreeClassifier(max_depth=10, min_samples_split=5)
dt.fit(X, y)
y_pred_dt = dt.predict(X_test)
print("Decision Trees:")
print(classification_report(y_test, y_pred_dt))


importances = dt.feature_importances_
toxic_features_dt = sorted(zip(importances, feature_names), reverse=True)[:5]
top_toxic_words['Decision Trees'] = [word for importance, word in toxic_features_dt]

# Naive Bayes
nb = MultinomialNB(alpha=1.0, force_alpha=True, fit_prior=False, class_prior=None)
nb.fit(X, y)
y_pred_nb = nb.predict(X_test)
y_proba_nb= nb.predict_proba(X_test)
print("Naive Bayes:")
print(classification_report(y_test, y_pred_nb))

word_vectors = vectorizer.transform(feature_names)
class_probas = nb.predict_proba(word_vectors)
importances = np.squeeze(class_probas[:, 1:], axis=1)
toxic_features_nb = sorted(zip(importances, feature_names), key=lambda x: x[0], reverse=True)[:5]
top_toxic_words['Naive Bayes'] = [word for log_prob, word in toxic_features_nb]

# Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X, y)
y_pred_rf = rf.predict(X_test)
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))


importances_rf = rf.feature_importances_
toxic_features_rf = sorted(zip(importances_rf, feature_names), reverse=True)[:5]
top_toxic_words['Random Forest'] = [word for importance, word in toxic_features_rf]


print("\nРезультаты:")
for clf_name, words in top_toxic_words.items():
    print(f"{clf_name}: {', '.join(words)}")

Logistic Regression:
              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88       953
         1.0       0.80      0.69      0.74       489

    accuracy                           0.84      1442
   macro avg       0.83      0.80      0.81      1442
weighted avg       0.83      0.84      0.83      1442

Decision Trees:
              precision    recall  f1-score   support

         0.0       0.69      0.98      0.81       953
         1.0       0.82      0.14      0.23       489

    accuracy                           0.70      1442
   macro avg       0.75      0.56      0.52      1442
weighted avg       0.73      0.70      0.62      1442

Naive Bayes:
              precision    recall  f1-score   support

         0.0       0.90      0.88      0.89       953
         1.0       0.77      0.81      0.79       489

    accuracy                           0.86      1442
   macro avg       0.84      0.84      0.84      1442
weighted avg       0.86 